In [3]:
import os
import torch
import torchvision

os.makedirs("modules")

FileExistsError: [Errno 17] File exists: 'modules'

>### data_preprocessing.py

In [4]:
%%writefile modules/data_preprocess.py
"""
creates dataoaders from dir
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS
):
    """creates train and test dataloaders

    Args:
        train_dir (str): _description_
        test_dir (str): _description_
        transform (transforms.Compose): _description_
        batch_size (int): _description_
        num_workers (int, optional): _description_. Defaults to NUM_WORKERS.

    Returns:
        Tuple(train_dataloader, test_dataloader, class_names): _description_
    """
    train_data = datasets.ImageFolder(
        root=train_dir,
        transform=transform,
        target_transform=None
    )
    
    test_data = datasets.ImageFolder(
        root=train_dir,
        transform=transform,
        target_transform=None
    )
    
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        shuffle=True
        )
    
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=NUM_WORKERS,
        shuffle=False)
    
    class_names = train_data.classes
    
    return train_dataloader, test_dataloader, class_names

Overwriting modules/data_preprocess.py


In [10]:
from pathlib import Path
train_dir = Path("data/pizza_steak_sushi/train")
test_dir = Path("data/pizza_steak_sushi/test")

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [5]:
from torchvision import transforms

In [6]:
data_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor
])

In [8]:
from modules.data_preprocess import create_dataloaders

In [12]:
train_dataloader, test_dataloader, class_anmes = create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=data_transform,
                                                                               batch_size=32
                                                                               )

In [6]:
%%writefile modules/build_model.py
"""
make model arch here
"""
import torch

from torch import nn

class TinyVGG(nn.Module):
    """creates tinyvgg
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        return self.classifier(self.block_2(self.block_1(x))) #leverages the benefits of operator fusion

Overwriting modules/build_model.py


In [5]:
%%writefile modules/train_engine.py
"""
training and testing model
"""
from typing import Dict, List, Tuple

import torch

from tqdm.auto import tqdm

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """trains models for a single epoch

    Args:
    model: .pth model
    dataloader: datalodaer
    loss_fn: nn.loss
    optimizer: torch.optim
    device: cpu or cuda

    Returns:
    Tuple(train_loss, train_acc)
    """

    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """test models for a single epoch

    Args:
    model: .pth model
    dataloader: datalodaer
    loss_fn: nn.loss
    optimizer: torch.optim
    device: cpu or cuda

    Returns:
    Tuple(test_loss, test_acc)
    """
    model.eval() 
    test_loss, test_acc = 0, 0
    with torch.inference_mode():

        for batch, (X, y) in enumerate(dataloader):

            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    """trains and tests model

    Args:
    model: .pth model
    train_dataloader: dataloader
    test_dataloader: dataloader
    loss_fn: nn.loss
    optimizer: torch.optim
    epochs: 
    device: cpu or cuda

    Returns:
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    """
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results


Overwriting modules/train_engine.py


In [ ]:
from modules import train_engine
train_engine.train()

TypeError: train() missing 7 required positional arguments: 'model', 'train_dataloader', 'test_dataloader', 'optimizer', 'loss_fn', 'epochs', and 'device'

In [7]:
%%writefile modules/utils.py 
"""
utils for saving
"""
from pathlib import Path

import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """saves .pth model to dir

    args:
    model: .pth model.
    target_dir: target_dir.
    model_name: "model_name.pth"

    example:
    save_model(model=model_0,
               target_dir="models",
               model_name="dummy_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing modules/utils.py


In [8]:
%%writefile modules/train.py
"""
trains a model
"""

import os
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils


# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="dummy_model.pth")


Overwriting modules/train.py


In [2]:
!python modules/train.py

  0%|                                                     | 0/5 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/home/stanley/Documents/shidder/ml/pytorching/pytorch_modules/modules/train.py", line 50, in <module>
    train_engine.train(model=model,
  File "/home/stanley/Documents/shidder/ml/pytorching/pytorch_modules/modules/train_engine.py", line 68, in train
    train_loss, train_acc = train_step(model=model,
                            ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/stanley/Documents/shidder/ml/pytorching/pytorch_modules/modules/train_engine.py", line 15, in train_step
    for batch, (X, y) in enumerate(dataloader):
  File "/usr/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1345, in _next_data
    return self._process_data(data)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.1